In [ ]:
# Step 1: Install required libraries
!pip install wikipedia-api langchain-community langchain-huggingface chromadb

In [3]:
# Step 2: Import necessary libraries
import wikipediaapi
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings as HFEmbeddings

class WikiQABot:

  def __init__(self, language = 'en'):
    # Initialise Wikipedia API

    self.wiki_wiki = wikipediaapi.Wikipedia('Project', language)
    self.docsearch = None

  def load_wikipedia_page(self, page_title):
    """ Load a Wikipedia page and return its text"""

    page = self.wiki_wiki.page(page_title)
    if not page.exists():
      print(f"The page '{page_title}' does not exist.")
      return None

    return page.text

  def setup_qa_bot(self, page_text):
    """ Create embeddings and setup the Q&A bot"""

    embeddings = HFEmbeddings(model_name="all-MiniLM-L6-v2")

    # Split text into chunks for processing
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_text(page_text)

    # Create a vector store for similarity search
    self.docsearch = Chroma.from_texts(
        texts, embeddings, metadatas=[{"source": str(i)} for
                                      i in range(len(texts))]
        )

  def query_bot(self):
    """ Interactively query the bot """

    while True:
      query = input("Type your question(or type 'exit' to quit):")
      if query.lower() == 'exit':
        break

      if self.docsearch is None:
        print("The Q&A bot is not set up yet. Please load a Wikipedia page first.")
        continue

      docs = self.docsearch.similarity_search_with_score(query, k=1)
      for doc in docs:
        print(f"Answer:{doc[0].page_content} (Source: {doc[0].metadata['source']})")

  def run(self):
    """ Main interactive loop to run the bot"""

    print("Welcome to the Wikipedia Q&A Bot:")
    page_title = input("Enter the title of the WIkipedia page you want to load:")

    # Load the specified wikipedia page
    page_text = self.load_wikipedia_page(page_title)
    if page_text:
      # Setup the Q&A bot with loaded text
      self.setup_qa_bot(page_text)

      # Start querying the bot
      self.query_bot()

In [ ]:
# Run the WikiQABot class
bot = WikiQABot()
bot.run()